In [1]:
from lmptools import Dump, DumpCallback, DumpFileIterator
from lmptools.exceptions import SkipSnapshot

In [2]:
class MyCallback(DumpCallback):
    def on_snapshot_parse_time(self, timestamp: int):
        if timestamp < 6000:
            raise SkipSnapshot(f"snap {timestamp} skipped")

In [7]:
d = Dump("dump.test.lammpstrj")

In [10]:
d.parse(callback = MyCallback())

<generator object Dump.parse at 0x7fe4bd5844a0>